In [1]:
#create a spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").\
                                     appName("spark_on_docker").\
                                     getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/13 00:24:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read.format("csv")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .load("work/TheDefinitiveGuide/Spark-The-Definitive-Guide/data/retail-data/all/*.csv")\
  .coalesce(5)
df.cache()
df.createOrReplaceTempView("dfTable")


In [3]:
df.show(5)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
only showing top 5 rows



In [4]:
df.schema   

StructType(List(StructField(InvoiceNo,StringType,true),StructField(StockCode,StringType,true),StructField(Description,StringType,true),StructField(Quantity,IntegerType,true),StructField(InvoiceDate,StringType,true),StructField(UnitPrice,DoubleType,true),StructField(CustomerID,IntegerType,true),StructField(Country,StringType,true)))

In [5]:
spark.sql("select * from dfTable").show(5)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
only showing top 5 rows



In [6]:
df.count()

541909

Aggregation Functions

count

In [7]:
from pyspark.sql.functions import count
df.select(count("InvoiceNo")).show() 

+----------------+
|count(InvoiceNo)|
+----------------+
|          541909|
+----------------+



In [8]:
spark.sql("select count(*), count(1), count('InvoiceNo') from dfTable").show()

+--------+--------+----------------+
|count(1)|count(1)|count(InvoiceNo)|
+--------+--------+----------------+
|  541909|  541909|          541909|
+--------+--------+----------------+



countDistinct

In [9]:
from pyspark.sql.functions import countDistinct

df.select(countDistinct("StockCode")).show() 

+-------------------------+
|count(DISTINCT StockCode)|
+-------------------------+
|                     4070|
+-------------------------+



In [10]:
spark.sql("select count(DISTINCT StockCode) from dfTable").show()

+-------------------------+
|count(DISTINCT StockCode)|
+-------------------------+
|                     4070|
+-------------------------+



approx_count_distinct

Often, we find ourselves working with large datasets and the exact distinct count is irrelevant.
There are times when an approximation to a certain degree of accuracy will work just fine, and
for that, you can use the approx_count_distinct function:

In [11]:
from pyspark.sql.functions import approx_count_distinct
df.select(\
    approx_count_distinct("StockCode").alias('1'), \
    approx_count_distinct("StockCode", 0.1).alias('2')).show() # 3364

21/12/13 00:25:14 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----+----+
|   1|   2|
+----+----+
|3804|3364|
+----+----+



In [12]:
spark.sql("select approx_count_distinct(StockCode, 0.1) from dfTable").show()

+--------------------------------+
|approx_count_distinct(StockCode)|
+--------------------------------+
|                            3364|
+--------------------------------+



first and last
You can get the first and last values from a DataFrame by using these two obviously named
functions. This will be based on the rows in the DataFrame, not on the values in the DataFrame:

In [13]:
from pyspark.sql.functions import first, last
df.select(first("StockCode"), last("StockCode")).show()

+----------------+---------------+
|first(StockCode)|last(StockCode)|
+----------------+---------------+
|          85123A|          22138|
+----------------+---------------+



In [14]:
spark.sql("SELECT first(StockCode), last(StockCode) FROM dfTable").show()

+----------------+---------------+
|first(StockCode)|last(StockCode)|
+----------------+---------------+
|          85123A|          22138|
+----------------+---------------+



min and max
To extract the minimum and maximum values from a DataFrame, use the min and max functions:

In [15]:
from pyspark.sql.functions import min, max
df.select(min("Quantity"), max("Quantity")).show()

+-------------+-------------+
|min(Quantity)|max(Quantity)|
+-------------+-------------+
|       -80995|        80995|
+-------------+-------------+



In [16]:
spark.sql("SELECT min(Quantity), max(Quantity) FROM dfTable").show()

+-------------+-------------+
|min(Quantity)|max(Quantity)|
+-------------+-------------+
|       -80995|        80995|
+-------------+-------------+



sum
Another simple task is to add all the values in a row using the sum function:

In [17]:
from pyspark.sql.functions import sum
df.select(sum("Quantity")).show() # 5176450

+-------------+
|sum(Quantity)|
+-------------+
|      5176450|
+-------------+



In [18]:
spark.sql("SELECT sum(Quantity) FROM dfTable").show()

+-------------+
|sum(Quantity)|
+-------------+
|      5176450|
+-------------+



sumDistinct

In addition to summing a total, you also can sum a distinct set of values by using the
sumDistinct function:

In [19]:
from pyspark.sql.functions import sum_distinct
df.select(sum_distinct("Quantity")).show() # 29310

+----------------------+
|sum(DISTINCT Quantity)|
+----------------------+
|                 29310|
+----------------------+



avg

Although you can calculate average by dividing sum by count, Spark provides an easier way to
get that value via the avg or mean functions. In this example, we use alias in order to more
easily reuse these columns later:

In [20]:
from pyspark.sql.functions import sum, count, avg, expr

df.select(
    count("Quantity").alias("total_transactions"),
    sum("Quantity").alias("total_purchases"),
    avg("Quantity").alias("avg_purchases"),
    expr("mean(Quantity)").alias("mean_purchases"))\
.selectExpr(
    "total_purchases/total_transactions",
    "avg_purchases",
    "mean_purchases"
).show()


+--------------------------------------+----------------+----------------+
|(total_purchases / total_transactions)|   avg_purchases|  mean_purchases|
+--------------------------------------+----------------+----------------+
|                      9.55224954743324|9.55224954743324|9.55224954743324|
+--------------------------------------+----------------+----------------+



In [21]:
df.select(
    (sum("Quantity")/count("Quantity")).alias("transactions quanty rate"),
    avg("Quantity").alias("avg_purchases"),
    expr("mean(Quantity)").alias("mean_purchases")).show()

+------------------------+----------------+----------------+
|transactions quanty rate|   avg_purchases|  mean_purchases|
+------------------------+----------------+----------------+
|        9.55224954743324|9.55224954743324|9.55224954743324|
+------------------------+----------------+----------------+



Variance and Standard Deviation

In [22]:
from pyspark.sql.functions import var_pop, stddev_pop
from pyspark.sql.functions import var_samp, stddev_samp

df.select(var_pop("Quantity"), var_samp("Quantity"),
stddev_pop("Quantity"), stddev_samp("Quantity")).show()

+------------------+------------------+--------------------+---------------------+
| var_pop(Quantity)|var_samp(Quantity)|stddev_pop(Quantity)|stddev_samp(Quantity)|
+------------------+------------------+--------------------+---------------------+
|47559.303646609056|47559.391409298754|  218.08095663447796|   218.08115785023418|
+------------------+------------------+--------------------+---------------------+



In [23]:
spark.sql("SELECT var_pop(Quantity), \
    var_samp(Quantity),stddev_pop(Quantity), \
    stddev_samp(Quantity) FROM dfTable").show()

+------------------+------------------+--------------------+---------------------+
| var_pop(Quantity)|var_samp(Quantity)|stddev_pop(Quantity)|stddev_samp(Quantity)|
+------------------+------------------+--------------------+---------------------+
|47559.303646609056|47559.391409298754|  218.08095663447796|   218.08115785023418|
+------------------+------------------+--------------------+---------------------+



skewness and kurtosis

In [24]:
from pyspark.sql.functions import skewness, kurtosis
df.select(skewness("Quantity"), kurtosis("Quantity")).show()

+-------------------+------------------+
| skewness(Quantity)|kurtosis(Quantity)|
+-------------------+------------------+
|-0.2640755761052562|119768.05495536952|
+-------------------+------------------+



In [25]:
spark.sql("SELECT skewness(Quantity), kurtosis(Quantity) FROM dfTable").show()

+-------------------+------------------+
| skewness(Quantity)|kurtosis(Quantity)|
+-------------------+------------------+
|-0.2640755761052562|119768.05495536952|
+-------------------+------------------+



Covariance and Correlation

In [26]:
from pyspark.sql.functions import corr, covar_pop, covar_samp
df.select(\
    corr("InvoiceNo", "Quantity"), \
    covar_samp("InvoiceNo", "Quantity"),\
    covar_pop("InvoiceNo", "Quantity")\
).show()

+-------------------------+-------------------------------+------------------------------+
|corr(InvoiceNo, Quantity)|covar_samp(InvoiceNo, Quantity)|covar_pop(InvoiceNo, Quantity)|
+-------------------------+-------------------------------+------------------------------+
|     4.912186085635685E-4|             1052.7280543902734|            1052.7260778741693|
+-------------------------+-------------------------------+------------------------------+



In [27]:
r = spark.sql("SELECT corr(InvoiceNo, Quantity), \
                  covar_samp(InvoiceNo, Quantity),\
                  covar_pop(InvoiceNo, Quantity)\
            FROM dfTable")
r.show()

+-------------------------+-------------------------------+------------------------------+
|corr(InvoiceNo, Quantity)|covar_samp(InvoiceNo, Quantity)|covar_pop(InvoiceNo, Quantity)|
+-------------------------+-------------------------------+------------------------------+
|     4.912186085635685E-4|             1052.7280543902734|            1052.7260778741693|
+-------------------------+-------------------------------+------------------------------+



Aggregating to Complex Types

In Spark, you can perform aggregations not just of numerical values using formulas, you can also
perform them on complex types.For example, we can collect a list of values present in a given
column or only the unique values by collecting to a set.

In [28]:
from pyspark.sql.functions import collect_set, collect_list
df.agg(collect_set("Country").alias("A"), collect_list("Country").alias("B")).selectExpr("A", "B").show()

+--------------------+--------------------+
|                   A|                   B|
+--------------------+--------------------+
|[Portugal, Italy,...|[United Kingdom, ...|
+--------------------+--------------------+



In [29]:
spark.sql("SELECT collect_set(Country), collect_set(Country) FROM dfTable").show()  


+--------------------+--------------------+
|collect_set(Country)|collect_set(Country)|
+--------------------+--------------------+
|[Portugal, Italy,...|[Portugal, Italy,...|
+--------------------+--------------------+



Grouping

Thus far, we have performed only DataFrame-level aggregations. A more common task is to
perform calculations based on groups in the data. This is typically done on categorical data for
which we group our data on one column and perform some calculations on the other columns
that end up in that group.

In [30]:
df.groupBy("InvoiceNo", "CustomerId").count().show()

+---------+----------+-----+
|InvoiceNo|CustomerId|count|
+---------+----------+-----+
|   536846|     14573|   76|
|   537026|     12395|   12|
|   537883|     14437|    5|
|   538068|     17978|   12|
|   538279|     14952|    7|
|   538800|     16458|   10|
|   538942|     17346|   12|
|  C539947|     13854|    1|
|   540096|     13253|   16|
|   540530|     14755|   27|
|   541225|     14099|   19|
|   541978|     13551|    4|
|   542093|     17677|   16|
|   537159|     14527|   28|
|   537213|     12748|    6|
|   538191|     15061|   16|
|  C539301|     13496|    1|
|   539391|     12417|   11|
|   539462|     14032|   15|
|  C539710|     14769|    2|
+---------+----------+-----+
only showing top 20 rows



In [31]:
spark.sql("SELECT InvoiceNo, CustomerId, count(*) FROM dfTable GROUP BY InvoiceNo, CustomerId").show() 

+---------+----------+--------+
|InvoiceNo|CustomerId|count(1)|
+---------+----------+--------+
|   536846|     14573|      76|
|   537026|     12395|      12|
|   537883|     14437|       5|
|   538068|     17978|      12|
|   538279|     14952|       7|
|   538800|     16458|      10|
|   538942|     17346|      12|
|  C539947|     13854|       1|
|   540096|     13253|      16|
|   540530|     14755|      27|
|   541225|     14099|      19|
|   541978|     13551|       4|
|   542093|     17677|      16|
|   537159|     14527|      28|
|   537213|     12748|       6|
|   538191|     15061|      16|
|  C539301|     13496|       1|
|   539391|     12417|      11|
|   539462|     14032|      15|
|  C539710|     14769|       2|
+---------+----------+--------+
only showing top 20 rows



Grouping with Expressions

In [32]:
from pyspark.sql.functions import count

df.groupBy("InvoiceNo").agg(
    count("Quantity").alias("quan"),
    expr("count(Quantity)")).show()

+---------+----+---------------+
|InvoiceNo|quan|count(Quantity)|
+---------+----+---------------+
|   536596|   6|              6|
|   536938|  14|             14|
|   537252|   1|              1|
|   537691|  20|             20|
|   538041|   1|              1|
|   538184|  26|             26|
|   538517|  53|             53|
|   538879|  19|             19|
|   539275|   6|              6|
|   539630|  12|             12|
|   540499|  24|             24|
|   540540|  22|             22|
|  C540850|   1|              1|
|   540976|  48|             48|
|   541432|   4|              4|
|   541518| 101|            101|
|   541783|  35|             35|
|   542026|   9|              9|
|   542375|   6|              6|
|   536597|  28|             28|
+---------+----+---------------+
only showing top 20 rows



Grouping with Expressions


In [33]:
df.groupBy("InvoiceNo", "CustomerId").agg(
    count("Quantity").alias("quan"),
    expr("count(Quantity)")).show()

+---------+----------+----+---------------+
|InvoiceNo|CustomerId|quan|count(Quantity)|
+---------+----------+----+---------------+
|   536846|     14573|  76|             76|
|   537026|     12395|  12|             12|
|   537883|     14437|   5|              5|
|   538068|     17978|  12|             12|
|   538279|     14952|   7|              7|
|   538800|     16458|  10|             10|
|   538942|     17346|  12|             12|
|  C539947|     13854|   1|              1|
|   540096|     13253|  16|             16|
|   540530|     14755|  27|             27|
|   541225|     14099|  19|             19|
|   541978|     13551|   4|              4|
|   542093|     17677|  16|             16|
|   537159|     14527|  28|             28|
|   537213|     12748|   6|              6|
|   538191|     15061|  16|             16|
|  C539301|     13496|   1|              1|
|   539391|     12417|  11|             11|
|   539462|     14032|  15|             15|
|  C539710|     14769|   2|     

Grouping with Maps

In [34]:
df.groupBy("InvoiceNo").agg(expr("avg(Quantity)"),expr("stddev_pop(Quantity)")).show()

+---------+------------------+--------------------+
|InvoiceNo|     avg(Quantity)|stddev_pop(Quantity)|
+---------+------------------+--------------------+
|   536596|               1.5|  1.1180339887498947|
|   536938|33.142857142857146|  20.698023172885524|
|   537252|              31.0|                 0.0|
|   537691|              8.15|   5.597097462078001|
|   538041|              30.0|                 0.0|
|   538184|12.076923076923077|   8.142590198943392|
|   538517|3.0377358490566038|  2.3946659604837897|
|   538879|21.157894736842106|  11.811070444356483|
|   539275|              26.0|  12.806248474865697|
|   539630|20.333333333333332|  10.225241100118645|
|   540499|              3.75|  2.6653642652865788|
|   540540|2.1363636363636362|  1.0572457590557278|
|  C540850|              -1.0|                 0.0|
|   540976|10.520833333333334|   6.496760677872902|
|   541432|             12.25|  10.825317547305483|
|   541518| 23.10891089108911|  20.550782784878713|
|   541783|1

In [35]:
spark.sql("SELECT InvoiceNo, avg(Quantity), stddev_pop(Quantity) FROM dfTable GROUP BY InvoiceNo").show()

+---------+------------------+--------------------+
|InvoiceNo|     avg(Quantity)|stddev_pop(Quantity)|
+---------+------------------+--------------------+
|   536596|               1.5|  1.1180339887498947|
|   536938|33.142857142857146|  20.698023172885524|
|   537252|              31.0|                 0.0|
|   537691|              8.15|   5.597097462078001|
|   538041|              30.0|                 0.0|
|   538184|12.076923076923077|   8.142590198943392|
|   538517|3.0377358490566038|  2.3946659604837897|
|   538879|21.157894736842106|  11.811070444356483|
|   539275|              26.0|  12.806248474865697|
|   539630|20.333333333333332|  10.225241100118645|
|   540499|              3.75|  2.6653642652865788|
|   540540|2.1363636363636362|  1.0572457590557278|
|  C540850|              -1.0|                 0.0|
|   540976|10.520833333333334|   6.496760677872902|
|   541432|             12.25|  10.825317547305483|
|   541518| 23.10891089108911|  20.550782784878713|
|   541783|1

Window Functions

A group-by takes data, and every row can go only into one grouping. A window function
calculates a return value for every input row of a table based on a group of rows, called a frame.
Each row can fall into one or more frames.

In [36]:
from pyspark.sql.functions import col, to_date
dfWithDate = df.withColumn("date", to_date(col("InvoiceDate"), "MM/d/yyyy H:mm"))
dfWithDate.createOrReplaceTempView("dfWithDate")

In [37]:
dfWithDate.select("*").show(2)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|      date|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|2010-12-01|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|2010-12-01|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+
only showing top 2 rows



first step to a window function is to create a window specification

In [38]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc

windowSpec = Window\
.partitionBy("CustomerId", "date")\
.orderBy(desc("Quantity"))\
.rowsBetween(Window.unboundedPreceding, Window.currentRow)

Now we want to use an aggregation function to learn more about each specific customer.
An example might be establishing the maximum purchase quantity over all time. To answer this, we
use the same aggregation functions that we saw earlier by passing a column name or expression.
In addition, we indicate the window specification that defines to which frames of data this
function will apply:

In [39]:
from pyspark.sql.functions import max

maxPurchaseQuantity = max(col("Quantity")).over(windowSpec)

You will notice that this returns a column (or expressions). We can now use this in a DataFrame
select statement. Before doing so, though, we will create the purchase quantity rank. To do that
we use the dense_rank function to determine which date had the maximum purchase quantity
for every customer. We use dense_rank as opposed to rank to avoid gaps in the ranking
sequence when there are tied values (or in our case, duplicate rows):

In [40]:
from pyspark.sql.functions import dense_rank, rank

purchaseDenseRank = dense_rank().over(windowSpec)
purchaseRank = rank().over(windowSpec)

This also returns a column that we can use in select statements. Now we can perform a select to
view the calculated window values

In [41]:
from pyspark.sql.functions import col
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

dfWithDate.where("CustomerId IS NOT NULL").orderBy("CustomerId")\
.select(
col("CustomerId"),
col("date"),
col("Quantity"),
purchaseRank.alias("quantityRank"),
purchaseDenseRank.alias("quantityDenseRank"),
maxPurchaseQuantity.alias("maxPurchaseQuantity")).show()


+----------+----------+--------+------------+-----------------+-------------------+
|CustomerId|      date|Quantity|quantityRank|quantityDenseRank|maxPurchaseQuantity|
+----------+----------+--------+------------+-----------------+-------------------+
|     12346|2011-01-18|   74215|           1|                1|              74215|
|     12346|2011-01-18|  -74215|           2|                2|              74215|
|     12347|2010-12-07|      36|           1|                1|                 36|
|     12347|2010-12-07|      30|           2|                2|                 36|
|     12347|2010-12-07|      24|           3|                3|                 36|
|     12347|2010-12-07|      12|           4|                4|                 36|
|     12347|2010-12-07|      12|           4|                4|                 36|
|     12347|2010-12-07|      12|           4|                4|                 36|
|     12347|2010-12-07|      12|           4|                4|             

In [42]:
spark.sql(" SELECT CustomerId, date, Quantity,\
                    rank(Quantity) OVER (PARTITION BY CustomerId, date\
                                            ORDER BY Quantity DESC NULLS LAST\
                                            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as rank,\
                    dense_rank(Quantity) OVER (PARTITION BY CustomerId, date\
                                            ORDER BY Quantity DESC NULLS LAST\
                                            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as dRank,\
                    max(Quantity) OVER (PARTITION BY CustomerId, date\
                                            ORDER BY Quantity DESC NULLS LAST\
                                            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as maxPurchase\
            FROM dfWithDate WHERE CustomerId IS NOT NULL ORDER BY CustomerId").show()

+----------+----------+--------+----+-----+-----------+
|CustomerId|      date|Quantity|rank|dRank|maxPurchase|
+----------+----------+--------+----+-----+-----------+
|     12346|2011-01-18|   74215|   1|    1|      74215|
|     12346|2011-01-18|  -74215|   2|    2|      74215|
|     12347|2010-12-07|      36|   1|    1|         36|
|     12347|2010-12-07|      30|   2|    2|         36|
|     12347|2010-12-07|      12|   4|    4|         36|
|     12347|2010-12-07|      12|   4|    4|         36|
|     12347|2010-12-07|      24|   3|    3|         36|
|     12347|2010-12-07|      12|   4|    4|         36|
|     12347|2010-12-07|      12|   4|    4|         36|
|     12347|2010-12-07|      12|   4|    4|         36|
|     12347|2010-12-07|      12|   4|    4|         36|
|     12347|2010-12-07|      12|   4|    4|         36|
|     12347|2010-12-07|      12|   4|    4|         36|
|     12347|2010-12-07|      12|   4|    4|         36|
|     12347|2010-12-07|      12|   4|    4|     

Grouping Sets

Sometimes we want something a bit more complete—an aggregation across multiple groups. 
We achieve this by using grouping sets.
Grouping sets are a low-level tool for combining sets o0f aggregations together. 
They give you the ability to create arbitrary aggregation in their group-by statements.

In [44]:
dfNoNull = dfWithDate.drop()
dfNoNull.createOrReplaceTempView("dfNoNull")

In [60]:
spark.sql("SELECT CustomerId, stockCode, sum(Quantity) \
            FROM dfNoNull\
            GROUP BY customerId, stockCode\
            ORDER BY CustomerId DESC, stockCode DESC").show(5)

+----------+---------+-------------+
|CustomerId|stockCode|sum(Quantity)|
+----------+---------+-------------+
|     18287|    85173|           48|
|     18287|   85040A|           48|
|     18287|   85039B|          120|
|     18287|   85039A|           96|
|     18287|    84920|            4|
+----------+---------+-------------+
only showing top 5 rows



In [63]:
spark.sql("\
  SELECT CustomerId, stockCode, sum(Quantity) \
    FROM dfNoNull \
GROUP BY customerId, stockCode GROUPING SETS((customerId, stockCode)) \
ORDER BY CustomerId DESC, stockCode DESC\
").show(5)

+----------+---------+-------------+
|customerId|stockCode|sum(Quantity)|
+----------+---------+-------------+
|     18287|    85173|           48|
|     18287|   85040A|           48|
|     18287|   85039B|          120|
|     18287|   85039A|           96|
|     18287|    84920|            4|
+----------+---------+-------------+
only showing top 5 rows



In [75]:
spark.sql("\
  SELECT CustomerId, stockCode, sum(Quantity) \
    FROM dfNoNull \
GROUP BY customerId, stockCode GROUPING SETS((customerId, stockCode),()) \
ORDER BY CustomerId DESC, stockCode DESC\
").show(5)

+----------+---------+-------------+
|customerId|stockCode|sum(Quantity)|
+----------+---------+-------------+
|     18287|    85173|           48|
|     18287|   85040A|           48|
|     18287|   85039B|          120|
|     18287|   85039A|           96|
|     18287|    84920|            4|
+----------+---------+-------------+
only showing top 5 rows



Rollups

Multidimensional aggregation that performs a variety of group-by style calculations for us.

In [77]:
rolledUpDF = dfNoNull.rollup("Date", "Country").agg(sum("Quantity"))\
.selectExpr("Date", "Country", "`sum(Quantity)` as total_quantity")\
.orderBy("Date")

rolledUpDF.show(5)

+----------+--------------+--------------+
|      Date|       Country|total_quantity|
+----------+--------------+--------------+
|      null|          null|       5176450|
|2010-12-01|        Norway|          1852|
|2010-12-01|        France|           449|
|2010-12-01|     Australia|           107|
|2010-12-01|United Kingdom|         23949|
+----------+--------------+--------------+
only showing top 5 rows



In [79]:
rolledUpDF.where("Country IS NULL").show(5)

+----------+-------+--------------+
|      Date|Country|total_quantity|
+----------+-------+--------------+
|      null|   null|       5176450|
|2010-12-01|   null|         26814|
|2010-12-02|   null|         21023|
|2010-12-03|   null|         14830|
|2010-12-05|   null|         16395|
+----------+-------+--------------+
only showing top 5 rows



In [80]:
rolledUpDF.where("Date IS NULL").show(5)

+----+-------+--------------+
|Date|Country|total_quantity|
+----+-------+--------------+
|null|   null|       5176450|
+----+-------+--------------+



Cube

A cube takes the rollup to a level deeper. Rather than treating elements hierarchically, a cube
does the same thing across all dimensions. This means that it won’t just go by date over the
entire time period, but also the country. To pose this as a question again, can you make a table
that includes the following?

    The total across all dates and countries
    The total for each date across all countries
    The total for each country on each date
    The total for each country across all dates

The method call is quite similar, but instead of calling rollup, we call cube:

In [81]:
from pyspark.sql.functions import sum

dfNoNull.cube("Date", "Country").agg(sum(col("Quantity")))\
.select("Date", "Country", "sum(Quantity)").orderBy("Date").show(5)

+----+---------+-------------+
|Date|  Country|sum(Quantity)|
+----+---------+-------------+
|null|  Finland|        10666|
|null|   Greece|         1556|
|null|Lithuania|          652|
|null|   Poland|         3653|
|null|  Iceland|         2458|
+----+---------+-------------+
only showing top 5 rows



Pivot

Pivots make it possible for you to convert a row into a column. 
For example, in our current data we have a Country column. With a pivot, 
we can aggregate according to some function for each of those given countries 
and display them in an easy-to-query way:

In [84]:
pivoted = dfWithDate.groupBy("date").pivot("Country").sum()

In [85]:
pivoted.where("date > '2011-12-05'").select("date" ,"`USA_sum(Quantity)`").show()

+----------+-----------------+
|      date|USA_sum(Quantity)|
+----------+-----------------+
|2011-12-06|             null|
|2011-12-09|             null|
|2011-12-08|             -196|
|2011-12-07|             null|
+----------+-----------------+

